In [7]:
# Hier berechne ich einen Mittelwert für jede Substitution

In [8]:
import pandas as pd
import data_cleanup as dc
import data_exploration as de

In [9]:
fpath = '../DMS_data/P53_HUMAN_Giacomelli_NULL_Etoposide_2018.csv'
df = pd.read_csv(fpath)


In [10]:
# Auftrennen der Mutante in alte und neue AS
mutations_df = dc.aufteilung_mut_pos(df)

In [19]:
# Gruppieren der Substitutionen, geordnet erst nach alter und dann nach neuer AS mit Inspiration von ChatGPT (.groupby in der Pandas Dokumentation beschreibt nur das groupen nach einem Parameter)
subs_df = mutations_df.groupby(["AS_old", "AS_new"])

# Berechnen des means für jede Kombination
mean_scores = subs_df.DMS_score.mean()
# Es gibt 380 Möglichkeiten, 20 AS mit Reihenfolge und ohne zurücklegen zu kombinieren. Aus den Mittelwerten dieser 380 möglichen Substitutionen soll nun eine Distanzmatrix erstellt werden.

In [32]:
# Da "mean_scores" eine Serie ist, muss diese erst wieder in ein Data Frame umgewandelt werden. Dafür müssen den einzelnen Zeilen wieder konkrete Indizes zugewiesen werden, damit wieder ein Data Frame entsteht.
mean_scores_df = mean_scores.reset_index()

# Pivot erstellt aus unserer in einem Data Frame aufgelisteten Substitutionen ein Data Frame, das in Form einer Tabelle eine Übersicht der Mittelwerte zeigt. Die Reihen sind die alten AS, die Spalten die neuen AS.
mean_substitutions = mean_scores_df.pivot(index="AS_old", columns="AS_new", values= "DMS_score")
dc.rmv_na(mean_substitutions)

print(mean_substitutions.columns)

Index(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'V', 'W', 'Y'],
      dtype='object', name='AS_new')


### Ergänzungen von Frido für Distanzmatrix

In [39]:
from sklearn.metrics.pairwise import euclidean_distances

# Speichern der "AS_new"-Spalte für Benennung der neuen Spalten
labels_column = 'AS_new'

# Transpose the DataFrame
mean_substitutions_t = mean_substitutions.T

# Berechnen der Distanzen der AA zueinander
dms_distances = euclidean_distances(mean_substitutions_t.values)

# Umwandeln zurück in einen DF zur besseren Übersicht
mean_substitutions_df = pd.DataFrame(dms_distances, index=mean_substitutions_t.index, columns=mean_substitutions_t.index)

# Print the first 20 rows of mean_substitutions
mean_substitutions_df.head(20)


AS_new,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
AS_new,,,,,,,,,,,,,,,,,,,,
A,0.000000,1.870447,2.057424,1.677953,2.074712,1.800098,1.616680,1.972078,1.663155,1.731438,2.464359,1.605218,2.599323,1.324979,1.734199,1.334446,1.719908,2.085699,1.770016,1.858589
C,1.870447,0.000000,2.932487,2.646722,1.435872,2.400857,2.179488,1.457241,2.341538,1.861285,1.543734,1.546109,3.430827,2.037637,2.128601,1.433622,1.808142,1.330392,2.086560,1.847943
D,2.057424,2.932487,0.000000,1.147741,2.851973,1.623479,1.562002,3.116516,1.675408,2.846908,3.331162,1.705688,1.851973,1.845346,1.894457,2.039805,2.226007,3.020563,1.912520,2.303482
E,1.677953,2.646722,1.147741,0.000000,2.716283,1.383653,1.591554,2.687395,1.266700,2.444568,3.072152,1.613702,1.688013,1.448361,1.697370,1.790569,1.874593,2.744142,1.711126,2.160131
F,2.074712,1.435872,2.851973,2.716283,0.000000,2.604831,1.940718,1.640068,2.634391,1.889827,1.739786,1.771891,3.318476,2.286170,2.310283,1.752132,2.095080,1.503799,1.734475,1.324595
G,1.800098,2.400857,1.623479,1.383653,2.604831,0.000000,1.812648,2.646116,1.241790,2.267662,2.792097,1.595640,1.669550,1.808504,1.539402,1.626238,1.924800,2.621017,1.822127,2.155682
H,1.616680,2.179488,1.562002,1.591554,1.940718,1.812648,0.000000,2.420617,1.625366,2.212610,2.622430,1.331438,2.415598,1.438266,1.746828,1.538785,1.981881,2.435773,1.111196,1.259381
I,1.972078,1.457241,3.116516,2.687395,1.640068,2.646116,2.420617,0.000000,2.528704,1.504370,1.357431,2.109293,3.210016,2.202334,2.317928,1.955986,1.579251,0.918103,2.373551,2.249416
K,1.663155,2.341538,1.675408,1.266700,2.634391,1.241790,1.625366,2.528704,0.000000,2.065978,2.711528,1.464649,2.097893,1.391438,1.339549,1.518363,1.716352,2.545937,1.814540,2.160693
